In [1]:
from numpy.random import seed
seed(101)
from tensorflow import set_random_seed
set_random_seed(101)

import pandas as pd
import numpy as np
#import keras
#from keras import backend as K

import tensorflow
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

import os


import itertools
import shutil
import matplotlib.pyplot as plt
%matplotlib inline


In [18]:
os.listdir('F:/CBIS-DDSM/compressed/MASS TRAINING/training')

base_dir = 'base_dir'
os.mkdir(base_dir)


In [19]:
# train_dir
train_dir = os.path.join(base_dir, 'train_dir')
os.mkdir(train_dir)

# val_dir
val_dir = os.path.join(base_dir, 'val_dir')
os.mkdir(val_dir)


In [20]:
# create new folders inside train_dir
MALIGNANT_calcification= os.path.join(train_dir, 'MALIGNANT_calcification')
os.mkdir(MALIGNANT_calcification)
BENIGN_calcification = os.path.join(train_dir, 'BENIGN_calcification')
os.mkdir(BENIGN_calcification)
BENIGN_mass= os.path.join(train_dir, 'BENIGN_mass')
os.mkdir(BENIGN_mass)
MALIGNANT_mass = os.path.join(train_dir, 'MALIGNANT_mass ')
os.mkdir(MALIGNANT_mass )



MALIGNANT_calcification= os.path.join(val_dir, 'MALIGNANT_calcification')
os.mkdir(MALIGNANT_calcification)
BENIGN_calcification = os.path.join(val_dir, 'BENIGN_calcification')
os.mkdir(BENIGN_calcification)

BENIGN_mass= os.path.join(val_dir, 'BENIGN_mass')
os.mkdir(BENIGN_mass)
MALIGNANT_mass = os.path.join(val_dir, 'MALIGNANT_mass ')
os.mkdir(MALIGNANT_mass )

In [168]:
df_data = pd.read_csv('F:/CBIS-DDSM/compressed/MASS TRAINING/training/train_label.csv')

df_data.head()

,Image_Name,Patient_ID,Breast_Density,Side_L_R,Image View,Abnormality_ID,Abnormality_Type,Mass_Shape,Mass_Margins,Assessment,Pathology,Subtlety,Class
0,P_00005_RIGHT_CC,P_00005,3,RIGHT,CC,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,MALIGNANT_calcification
1,P_00005_RIGHT_MLO,P_00005,3,RIGHT,MLO,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,MALIGNANT_calcification
2,P_00007_LEFT_CC,P_00007,4,LEFT,CC,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,BENIGN_calcification
3,P_00007_LEFT_MLO,P_00007,4,LEFT,MLO,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,BENIGN_calcification
4,P_00008_LEFT_CC,P_00008,1,LEFT,CC,1,calcification,NaN,REGIONAL,2,BENIGN,3,BENIGN_calcification


In [169]:
# this will tell us how many images are associated with each lesion_id
df = df_data.groupby('Image_Name').count()

# now we filter out lesion_id's that have only one image associated with it
df = df[df['Patient_ID'] == 1]

df.reset_index(inplace=True)

df.head()

,Image_Name,Patient_ID,Breast_Density,Side_L_R,Image View,Abnormality_ID,Abnormality_Type,Mass_Shape,Mass_Margins,Assessment,Pathology,Subtlety,Class
0,P_00001_LEFT_CC,1,1,1,1,1,1,1,1,1,1,1,1
1,P_00001_LEFT_MLO,1,1,1,1,1,1,1,1,1,1,1,1
2,P_00004_LEFT_CC,1,1,1,1,1,1,1,1,1,1,1,1
3,P_00004_LEFT_MLO,1,1,1,1,1,1,1,1,1,1,1,1
4,P_00004_RIGHT_MLO,1,1,1,1,1,1,1,1,1,1,1,1


In [170]:
# here we identify lesion_id's that have duplicate images and those that have only
# one image.

def identify_duplicates(x):
    
    unique_list = list(df['Image_Name'])
    
    if x in unique_list:
        return 'no_duplicates'
    else:
        return 'has_duplicates'
    
# create a new colum that is a copy of the lesion_id column
df_data['duplicates'] = df_data['Image_Name']
# apply the function to this new column
df_data['duplicates'] = df_data['duplicates'].apply(identify_duplicates)

df_data.head()

,Image_Name,Patient_ID,Breast_Density,Side_L_R,Image View,Abnormality_ID,Abnormality_Type,Mass_Shape,Mass_Margins,Assessment,Pathology,Subtlety,Class,duplicates
0,P_00005_RIGHT_CC,P_00005,3,RIGHT,CC,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,MALIGNANT_calcification,no_duplicates
1,P_00005_RIGHT_MLO,P_00005,3,RIGHT,MLO,1,calcification,AMORPHOUS,CLUSTERED,3,MALIGNANT,3,MALIGNANT_calcification,no_duplicates
2,P_00007_LEFT_CC,P_00007,4,LEFT,CC,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,BENIGN_calcification,no_duplicates
3,P_00007_LEFT_MLO,P_00007,4,LEFT,MLO,1,calcification,PLEOMORPHIC,LINEAR,4,BENIGN,4,BENIGN_calcification,no_duplicates
4,P_00008_LEFT_CC,P_00008,1,LEFT,CC,1,calcification,NaN,REGIONAL,2,BENIGN,3,BENIGN_calcification,no_duplicates


In [171]:
df_data['duplicates'].value_counts()


no_duplicates     2233
has_duplicates     478
Name: duplicates, dtype: int64

In [172]:
# now we filter out images that don't have duplicates
df = df_data[df_data['duplicates'] == 'no_duplicates']

df.shape

(2233, 14)

In [173]:
# now we create a val set using df because we are sure that none of these images
# have augmented duplicates in the train set
y = df['Class']

_, df_val = train_test_split(df, test_size=0.20, random_state=101, stratify=y)

df_val.shape

(447, 14)

In [174]:
df_val['Class'].value_counts()


BENIGN_calcification       121
BENIGN_mass                118
MALIGNANT_mass             113
MALIGNANT_calcification     95
Name: Class, dtype: int64

In [175]:
# This set will be df_data excluding all rows that are in the val set

# This function identifies if an image is part of the train
# or val set.
def identify_val_rows(x):
    # create a list of all the lesion_id's in the val set
    val_list = list(df_val['Image_Name'])
    
    if str(x) in val_list:
        return 'val'
    else:
        return 'train'

# identify train and val rows

# create a new colum that is a copy of the image_id column
df_data['train_or_val'] = df_data['Image_Name']
# apply the function to this new column
df_data['train_or_val'] = df_data['train_or_val'].apply(identify_val_rows)
   
# filter out train rows
df_train = df_data[df_data['train_or_val'] == 'train']


print(len(df_train))
print(len(df_val))

2264
447


In [176]:
df_train['Class'].value_counts()


BENIGN_calcification       764
BENIGN_mass                557
MALIGNANT_mass             514
MALIGNANT_calcification    429
Name: Class, dtype: int64

In [177]:
df_val['Class'].value_counts()


BENIGN_calcification       121
BENIGN_mass                118
MALIGNANT_mass             113
MALIGNANT_calcification     95
Name: Class, dtype: int64

In [178]:
# Set the image_id as the index in df_data
df_data.set_index('Image_Name', inplace=True)

In [210]:
# Get a list of images in each of the two folders
folder_1 = os.listdir('F:/CBIS-DDSM/compressed/MASS TRAINING/training/calc_training image')
folder_2 = os.listdir('F:/CBIS-DDSM/compressed/MASS TRAINING/training/mass-training image')

# Get a list of train and val images
train_list = list(df_train['Image_Name'])
val_list = list(df_val['Image_Name'])



# Transfer the train images

for image in train_list:
    
    fname = image + '.png'
    label = df_data.loc[image,'Class']
    
    if fname in folder_1:
        # source path to image
        src = os.path.join('F:/CBIS-DDSM/compressed/MASS TRAINING/training/calc_training image', fname)
        # destination path to image
        dst = os.path.join(train_dir,str(label), str(fname))
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)

    if fname in folder_2:
        # source path to image
        src = os.path.join('F:/CBIS-DDSM/compressed/MASS TRAINING/training/mass-training image', fname)
        # destination path to image
        dst = os.path.join(train_dir,str(label), str(fname)) 
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)


# Transfer the val images

for image in val_list:
    
    fname = image + '.png'
    label = df_data.loc[image,'Class']
    
    if fname in folder_1:
        # source path to image
        src = os.path.join('F:/CBIS-DDSM/compressed/MASS TRAINING/training/calc_training image', fname)
        # destination path to image
        dst = os.path.join(val_dir, str(label), str(fname))
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)

    if fname in folder_2:
        # source path to image
        src = os.path.join('F:/CBIS-DDSM/compressed/MASS TRAINING/training/mass-training image', fname)
        # destination path to image
        dst = os.path.join(val_dir, str(label), str(fname))
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)

In [211]:
print(len(os.listdir('base_dir/train_dir/BENIGN_calcification')))
print(len(os.listdir('base_dir/train_dir/MALIGNANT_calcification')))
print(len(os.listdir('base_dir/train_dir/BENIGN_mass')))
print(len(os.listdir('base_dir/train_dir/MALIGNANT_mass')))


353
240
470
448


In [206]:
print(len(os.listdir('base_dir/val_dir/BENIGN_calcification')))
print(len(os.listdir('base_dir/val_dir/MALIGNANT_calcification')))
print(len(os.listdir('base_dir/val_dir/BENIGN_mass')))
print(len(os.listdir('base_dir/val_dir/MALIGNANT_mass')))


91
65
117
111


In [4]:
class_list = ['BENIGN_calcification','BENIGN_mass','MALIGNANT_mass','MALIGNANT_calcification']


for item in class_list:
    
    # We are creating temporary directories here because we delete these directories later
    # create a base dir
    aug_dir = 'D:/Breast Cancer Grad.Proj/augm_dir'
    os.mkdir(aug_dir)
    # create a dir within the base dir to store images of the same class
    img_dir = os.path.join(aug_dir, 'img_dir')
    os.mkdir(img_dir)

    # Choose a class
    img_class = item

    # list all images in that directory
    img_list = os.listdir('D:/Breast Cancer Grad.Proj/whole image/train_dir/' + img_class)

    # Copy images from the class train dir to the img_dir e.g. class 'mel'
    for fname in img_list:
            # source path to image
            src = os.path.join('D:/Breast Cancer Grad.Proj/whole image/train_dir/' + img_class, fname)
            # destination path to image
            dst = os.path.join(img_dir, fname)
            # copy the image from the source to the destination
            shutil.copyfile(src, dst)


    # point to a dir containing the images and not to the images themselves
    path = aug_dir
    save_path = 'D:/Breast Cancer Grad.Proj/whole image/train_dir/' + img_class

    # Create a data generator
    datagen = ImageDataGenerator(
        rotation_range = 25.,
        shear_range = .2 ,
        zoom_range = [.8, 1.2],
        horizontal_flip=True,
        vertical_flip=True,
    )

    batch_size = 64

    aug_datagen = datagen.flow_from_directory(path,
                                           save_to_dir=save_path,
                                           save_format='jpg',
                                        
                                                    target_size=(224,224),
                                                    batch_size=batch_size)



    # Generate the augmented images and add them to the training folders
    
    ###########
    
    num_aug_images_wanted = 6000 # total number of images we want to have in each class
    
    ###########
    
    num_files = len(os.listdir(img_dir))
    num_batches = int(np.ceil((num_aug_images_wanted-num_files)/batch_size))

    # run the generator and create about 6000 augmented images
    for i in range(0,num_batches):

        imgs, labels = next(aug_datagen)
        
    # delete temporary directory with the raw image files
    shutil.rmtree('augm_dir')

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'D:/Breast Cancer Grad.Proj/augm_dir'